In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch.nn.functional import softmax, sigmoid
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/Users/taeheon/miniforge3/envs/stock_price/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. KR-FinBERT-SC 모델 불러오기
- 다중 분류 : 부정, 중립, 긍정

In [2]:
# 모델 이름
MODEL_NAME = "snunlp/KR-FinBert-SC"

# tokenizer initialize
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 사전학습된 감성 분류 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

print(model.config.num_labels)
# 추론 모드로 전환
model.eval()

3


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(20000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# 2. 감성 분석 함수 정의

In [4]:
# 문장(헤드라인) 입력 -> 3개의 감성 확률 벡터 반환
def analyze_sentiment(text : str) :
    # 입력 문장 토크나이징 및 텐서 변환
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # 추론 모드에서 모델 실행 (기울기 미계산)
    with torch.no_grad() :
        outputs = model(**inputs)

    # softmax를 통해 확률값 벡터화
    probs = softmax(outputs.logits, dim=1).squeeze().tolist()

    # 결과를 딕셔너리로 반환 (부정/중립/긍정 확률)
    return {
        'negative' : round(probs[0], 3), 
        'neutral' : round(probs[1], 3),
        'positive' : round(probs[2], 3),
    }

#### test

In [7]:
# 다중 분류
tmp = '삼성전자, 지난해 연 매출 300조 돌파…영업익 16% 감소'
result = analyze_sentiment(tmp)

print(result)

{'negative': 1.0, 'neutral': 0.0, 'positive': 0.0}


In [8]:
date_filter = "2025-05-20"

date_name = date_filter.split('-')[0][2:] + date_filter.split('-')[1] + date_filter.split('-')[2]

date_name

'250520'

# 3. CSV 불러와서 분석 및 일별 평균 계산

In [ ]:
# headline csv
df = pd.read_csv(f"{date_name}_headlines_2.csv", index_col=0)

df = df[df['news_count'] >= 50]

df = df[['date', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
         '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
         '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
         '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
         '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', 'news_count']]

# news_count 개수 = 50
df['news_count'] = 50

sentiment_daily = []

# 전체 날짜 순회
for idx, row in tqdm(df.iterrows(), total=len(df)):
    date = row['date']
    news_count = row['news_count']
    
    # 기사 컬럼 자동 추출 (숫자 문자열만)
    article_cols = [col for col in row.index if col.isdigit()]
    
    daily_vectors = []

    for col in article_cols:
        headline = row[col]
        if isinstance(headline, str) and len(headline.strip()) > 0:
            sentiment = analyze_sentiment(headline)
            daily_vectors.append([
                sentiment['negative'],
                sentiment['neutral'],
                sentiment['positive']
            ])

    # 기본 딕셔너리 구성
    sentiment_record = {
        'date': date,
        'news_count': news_count,
        **{col: row[col] for col in article_cols}  # 기사 원문도 저장
    }

    # 감성 벡터 평균 계산
    if daily_vectors:
        arr = np.array(daily_vectors)
        avg_vector = arr.mean(axis=0)
    else:
        # 뉴스가 하나도 없을 경우 → 중립
        avg_vector = [0.0, 1.0, 0.0]

    sentiment_record.update({
        'avg_negative': round(avg_vector[0], 4),
        'avg_neutral': round(avg_vector[1], 4),
        'avg_positive': round(avg_vector[2], 4)
    })

    sentiment_daily.append(sentiment_record)

100%|██████████| 440/440 [33:36<00:00,  4.58s/it]


# 4. 결과 저장 (sentiment_daily.csv)

In [13]:
df_sentiment = pd.DataFrame(sentiment_daily)

df_sentiment.head()

,date,news_count,1,2,3,4,5,6,7,8,...,44,45,46,47,48,49,50,avg_negative,avg_neutral,avg_positive
0,2024-01-01,115,‘금리 안정’ 기대감 부푼 증시… ‘반·아·바’ 주목 ‘소·유·건’ 주의 |||,삼성전자 연일 신고가 행진… ‘10만전자’ 시대 다시 오나 ||| 코스피 대장주인 ...,2세 이하 자녀 가구에 아파트 특별공급… ‘천원 아침밥’ 전국으로 [새해 이렇게 달...,[2024 산업전망] AI發 메모리 훈풍 채비 반도체업계…AI 심은 新가전 전쟁 |||,"[2024 산업전망] ‘청룡의 해’ 맞는 재계 총수들, 도전하는 리더가 주도한다 |||",10년간 버티니…삼성·LG의 ‘오리’가 ‘백조’됐다 ||| 전자업계가 글로벌 경기 ...,“해외서 세금아끼면 국내서 토해내라”…삼성 등 300곳 ‘최저한세 폭탄’ ||| 올...,"이상일 “2024년 갑진년 용의 해, 값진 용인의 해 되도록 할 것” ||| 이 시...",...,"9주 연속 상승한 코스피, 변동성 확대 경계 [증시풍향계] ||| 주식시장이 9주 ...",[모색(摸索) 2024] 금리인하 눈앞에…코스피 3000시대 열리나 |||,"새해 맞은 4대 그룹 총수, '새먹거리' 찾아 뛴다 ||| 삼성, SK, 현대차, ...",'미등기' 대기업총수 4명 보수 총 356억…'221억 회장님' 누구 ||| 국내 ...,'미등기' 대기업총수 4명 보수 年356억…'221억' 누구? ||| CJ 제공 C...,"삼성전자, AI 스마트폰 '갤럭시 S24' 출시…오는 17일 미국서 공개 ||| 핵...",“올해도 주식 ‘될놈될’ 예상…이 ETF로 반도체·2차전지 한 방에 투자하세요” [...,0.0808,0.8339,0.0852
1,2024-01-02,286,"上上, 현실이 되나 ||| 증권가는 이미 올해 코스피에 대한 장밋빛 전망이 한창이다...",‘폰 삼국지’ 15년 구도 재편되나구글·MS까지 뛰어든 AI폰 전쟁 |||,‘미등기 임원’ 대기업 총수 4명…2022년 보수로 356억원 받았다 ||| 국내 ...,“뭉쳐야 산다”… 경제 위기 묘수 찾아 머리 맞대는 대기업·中企 ||| 한국 경제에...,"[김경식의 퍼스펙티브] ESG, 도입은 했지만 인식 부족하고 제도도 미비했다 |||",16조 쓸어담았다… 작년 외국인 순매수 1위 종목은 ||| 외국인 투자자가 4년 만...,"‘AI와 삶의 질’ CES 화두… 기조연설 주연, 테크 아닌 화장품-유통 ||| 9...",[일사일언] ‘희망의 아이콘’ 리사 ||| 세밑을 눈앞에 두고 있던 지난달 한국과 ...,...,"이재용, 최태원, 정의선, 구광모 새해 첫 회동… '신년인사회' 참석 ||| 202...",주식 양도세 완화하자 개미들이 ‘풀매수’한 종목은? ||| 정부의 주식 양도소득세 ...,"""올해 CES 핵심 테마는 AI…7년새 시장 47배 급성장할 것""-KB ||| 이달...",[아주증시포커스] 외국인 '픽'은 반도체·바이오…매수세 몰린 이유는? 外 ||| 사...,초봉 400만원 낮아도…삼성전자 대신 택한 '1등 직장' ||| 취업준비생이 가장 ...,"[리포트 브리핑]삼성전자, '불리한 카운트에서의 전력투구' 목표가 95,000원 -...",하이證 “올해 반도체 강세 전망 맞다면 코스피 2900 상향” ||| 하이투자증권은...,0.1108,0.7076,0.1816
2,2024-01-03,262,"尹 ""새해 경제계‧정부 '원팀 코리아'로 더 힘차게 뛰자"" |||",“인구구조 변화 대비” “AI시대 퍼스트 무버” ||| ‘경쟁력을 확보하고’ ‘고객...,尹대통령 “경제 활력 체감하게 온 힘 쏟겠다” 최태원 “들숨 크게 마시고 힘차게 달...,"“금융투자세 폐지 K디스카운트 해소” 윤 대통령, 증시개장식 사상 첫 참석 |||",“초불확실성 위기… 도전·혁신·기술로 돌파해야 산다” |||,삼성 ‘첫 AI 비전’ LG ‘모빌리티’ SK ‘원더랜드’… 더 웅장해진 CES |||,“초불확실성 시대 진입… 기회의 원년 삼자” |||,대-중소기업 함께한 경제계 신년 인사회… “소외되는 이 없이 온기 느끼게 협력 강화...,...,"석유 부국 '사우디', 풍력 강국 '영국'도 K-탄소중립 올라 탄다 ||| [머니투...","[fn마켓워치]더인벤션랩, 2023년 초기창업기업 37개 투자 ||| [파이낸셜뉴스...",[CES 미리보기]산업 핵심 떠오른 'AI'…국내 기업 앞장선다 |||,“삼성전자 손절 고민하던 어머니…이젠 수익권 넘었다고 추매한다는데 어쩌죠” [투자3...,[CES D-7]'전 산업의 인공지능화'가 현실로…피부로 느끼는 스마트홈·초연결 |||,"30대 그룹 사장단 승진자, 전년대비 소폭 늘었다...여성은 0 ||| [사진=리덕...","""갤럭시S24 공개 눈앞…온디바이스 AI 성장 수혜株는?"" ||| 갤럭시S24 예상...",0.0806,0.7525,0.1669
3,2024-01-04,245,애플 “정당한 영업… 소송 불사”… 마시모 “손해나도 끝까지 간다” |||,"[Biz & Now] 웅진씽크빅, 신임 대표에 이봉주 전 삼성전자 부사장 |||",삼성 반도체 기술 中에 빼돌린 2명 기소… 2조3000억 피해 ||| 검찰이 중국 ...,갤럭시 첫 AI 온다 ||| “갤럭시 인공지능(AI)이 온다(Galaxy AI is...,"SK하이닉스, CES서 캐리커처 그려주는 ‘AI 포춘텔러’ 선보여 ||| SK하이닉...","삼성전자, CES서 빛 반사 줄인 OLED 게이밍 모니터 공개 ||| 삼성전자는 9...","“레고 조립과 같은 반도체 설계, 저희는 블록 뭉치를 드립니다” ||| “지금까지 ...",올해 CES 주연은 자동차… MS 등 빅테크도 ‘모빌리티 경쟁’ ||| 세계 최대 ...,...,"[오늘의 증시 리포트] 삼성전자, 4분기 DRAM 흑자전환 예상 ||| [이투데이]...",40만원대 삼성 5G폰 나온다…'갤럭시 A25' 국내 출시 ||| 삼성전자가 오는 ...,"삼성전자, 갤럭시 A25 국내 출시.. 44.9만원 ||| 삼성전자(005930)가...",‘갤럭시 A25 5G’ 출시...6.5형 120Hz 슈퍼 아몰레드 ||| 6.5형 ...,"삼성 중저가 폰, 갤럭시A25 출시…가격은 44만9900원 ||| 삼성전자 중저가 ...","[클릭 e종목]""퓨릿, 반도체향 소재 매출 증가 기대"" ||| SK증권은 퓨릿에 대...",롯데하이마트 1월 ‘메가 세일’…“이사하면 최대 20만원 쏩니다” ||| [헤럴드경...,0.0639,0.5750,0.3611
4,2024-01-05,204,4대 그룹 새 임원들의 원픽은…역시 ‘그랜저’ ||| 연초마다 국내 법인차 시장은 ...,"[사설] 반도체·잠수함 기술 잇단 유출, 경제안보 차원서 엄중 처벌해야 |||",“만지면 느낀다”…촉감까지 더한 완벽한 가상현실 올해 나온다는데 ||| 신경망 자극...,이재용·정의선 ‘모빌리티 동맹’… IoT 영역으로 확장 |||,"대법, 안철상·민유숙 후임 대법관 후보 42명 공개…여성은 7명 ||| 대법원은 지...","美, 마이크로칩에 보조금 2125억원...삼성·SK하이닉스도 '관심' ||| [휴스...","구글-아마존, CES서 AI 모빌리티 각축전… 인텔-퀄컴, AI칩 공개 |||",홍해 막혀 삼성-LG 물류비 먹구름… 유가 하루새 3% 상승 |||,...,"[리포트 브리핑]삼성전자, '메모리 중심의 실적 개선 지속' 목표가 100,000원...","용인시, 2023년 대한민국 지역경제대상 ‘2관왕’ ||| 한국지역경제학회 주관…지...",[CES ’24] 삼성전자-테슬라 “스마트싱스 에너지 협업” ||| 삼성 스마트싱스...,"[리포트 브리핑]삼성전자, '4Q23 Preview: 깔끔한 마무리' 목표가 95,...","[클릭 e종목]""삼성전자 업황개선·반도체값 상승…목표가↑"" ||| NH투자증권은 5...","‘아이폰, 韓서만 난리?’ 애플, 또 투자 등급 하향…美 나스닥·반도체지수 하락에 ...",삼성 '스마트싱스'로 테슬라 車 전력량 모니터링·제어한다 ||| [서울=뉴스핌] 이...,0.1070,0.5768,0.3161


In [ ]:
df_sentiment.to_csv(f"/Users/taeheon/stock_price/data/{date_name}_sentiment_daily.csv", index=False)